In [ ]:
!pip install ctransformers #transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 22.5 MB/s eta 0:00:00


In [ ]:
from ctransformers import AutoModelForCausalLM
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model = AutoModelForCausalLM.from_pretrained("gpt2")

llm = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7B-Chat-GGUF",
                                          model_file="llama-2-7b-chat.Q4_K_M.gguf",
                                          gpu_layers=0)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

In [ ]:
text = """
scale, high-quality annotated data.
To this end, we co-
developed FLD-5B that consists of 5.4 billion comprehen-
sive visual annotations on 126 million images, using an it-
erative strategy of automated image annotation and model
refinement. We adopted a sequence-to-sequence structure to
train Florence-2 to perform versatile and comprehensive vi-
sion tasks. Extensive evaluations on numerous tasks demon-
strated Florence-2 to be a strong vision foundation model
contender with unprecedented zero-shot and fine-tuning ca-
pabilities.
1. Introduction
In the realm of Artificial General Intelligence (AGI) sys-
tems, there has been a notable shift towards utilizing pre-
trained, versatile representations, acknowledged for task-
agnostic benefits accross diverse applications. This trend
is evident in natural language processing (NLP), where ad-
vanced models [5, 6, 19, 43, 65, 66] show adaptability with
comprehensive knowledge spanning various domains and
"""

In [ ]:
text = text.replace('-\n','')

In [ ]:
prompt = \
f"""
Instruction: Understand the below given input text and generate question and answer pairs.\n
Input:\n
{text}\n
Output:\n
"""
print(prompt)


Instruction: Understand the below given input text and generate question and answer pairs.

Input:


scale, high-quality annotated data.
To this end, we codeveloped FLD-5B that consists of 5.4 billion comprehensive visual annotations on 126 million images, using an iterative strategy of automated image annotation and model
refinement. We adopted a sequence-to-sequence structure to
train Florence-2 to perform versatile and comprehensive vision tasks. Extensive evaluations on numerous tasks demonstrated Florence-2 to be a strong vision foundation model
contender with unprecedented zero-shot and fine-tuning capabilities.
1. Introduction
In the realm of Artificial General Intelligence (AGI) systems, there has been a notable shift towards utilizing pretrained, versatile representations, acknowledged for taskagnostic benefits accross diverse applications. This trend
is evident in natural language processing (NLP), where advanced models [5, 6, 19, 43, 65, 66] show adaptability with
comprehen

In [ ]:
%%time
response = llm(prompt)
print(response)

Question 1: What is the main focus of the passage?
Answer: The main focus of the passage is on the development of a strong vision foundation model called Florence-2 that can perform diverse and comprehensive vision tasks.

Question 2: How was Florence-2 developed?
Answer: Florence-2 was developed using an iterative strategy of automated image annotation and model refinement, resulting in 5.4 billion comprehensive visual annotations on 126 million images.

Question 3: What is the purpose of using pretrained representations in AGI systems?
Answer: The purpose of using pretrained representations in AGI systems is to leverage task-agnostic benefits across diverse applications, which is evident in natural language processing (NLP) where advanced models show adaptability with comprehensive knowledge spanning various domains.
CPU times: user 8min 53s, sys: 713 ms, total: 8min 54s
Wall time: 5min 30s


In [ ]:
# inputs = tokenizer(prompt, return_tensors="pt")
# tokens = model.generate(**inputs, max_new_tokens=512)
# tokenizer.decode(tokens[0])

In [ ]:
import pandas as pd
from tqdm import tqdm

#initialize a dictionary for saving questions and answers
qa_dict = {'Question': [], 'Answer': []}

#read qa data file
df = pd.read_csv("qa.csv")

for row in tqdm(df.iterrows(), 'processing'):
  text = row[1].values[0]
  # print(text)

  # Split the input string into individual question-answer pairs
  qa_pairs = [pair.strip() for pair in text.split('\n') if pair.strip()]
  # print(qa_pairs)

  questions = []
  answers = []

  for pair in qa_pairs:
    # print(pair)
    #separate question and answer
    if pair.endswith('?'):
      questions.append(pair)
    else:
      pair = pair.replace("=-", "").replace("- ", "")
      answers.append(pair)

  if len(questions)!=len(answers):
    # print(text)
    continue

  else:
    # Append to the dictionary
    for question in questions:
      qa_dict['Question'].append(question)
    for answer in answers:
      qa_dict['Answer'].append(answer)

# Create a DataFrame
df_qa = pd.DataFrame(qa_dict)
# print(df_qa)
# break

processing: 132it [00:00, 13346.39it/s]


In [ ]:
df_qa.shape

(321, 2)

In [ ]:
df_qa.head()

,Question,Answer
0,1. What is the name of the new vision foundati...,Florence-2
1,2. What is the main objective of the Florence-...,To perform a diversity of tasks with simple in...
2,3. How does the Florence-2 model take user ins...,The Florence-2 model takes text-prompt as task...
3,4. What is the output format of the tasks that...,The output format of the tasks that the Floren...
4,5. Which annotation process do you use to gene...,The annotation process used in the paper is no...


In [ ]:
df_qa.to_csv("qna_data.csv", index=False)